## Make the dataframe including the percentage(%) of soil moisture for the each years and regions
- Date : Mar 22, 2023
- Author : JaeEun Yoo

In [1]:
import os
import glob
import requests
import json
import pandas as pd
from urllib.request import urlopen
from itertools import repeat
import xmltodict

In [2]:
regions = pd.read_csv('../샘플데이터/06. 토양수분_농업날씨365(open aPI)/관측지점.csv', encoding='cp949')

In [3]:
regions

,일련번호,도구분,지점코드,지점명,위도,경도,고도,설치주소,관측 시작일
0,1,강원도,024063D006,철원군 근남면,38.16262,127.32201,278.0,강원도 철원군 근남면 마현1길 20-41,2017-08-07
1,2,강원도,026002D003,태백시 매봉산,37.22308,128.96514,1153.0,강원도 태백시 매봉산길304,2015-12-16
2,3,강원도,026136A003,정선군 신동읍,37.25987,128.58477,261.0,강원도 정선군 신동읍 덕천리 산137,2015-10-19
3,4,강원도,200821A001,춘천시 신북읍,37.93400,127.75377,98.0,강원도 춘천시 신북읍 신북로 102,2010-09-28
4,5,강원도,209802A001,화천군 화천읍,38.11013,127.69448,139.0,강원도 화천군 화천읍 상리 531 화천군농업기술센터 실습,2011-02-22
...,...,...,...,...,...,...,...,...,...
194,207,충청북도,365803A001,진천군 진천읍,36.85385,127.42977,101.0,충북 진천군 진천읍 문진로 1433-6(교성리 313-3),2010-11-18
195,208,충청북도,370853A001,영동군 심천면,36.20687,127.71638,100.0,충북 영동군 심천면 약목리 282번지 실증시험포,2010-09-28
196,209,충청북도,373805A001,옥천군 옥천읍,36.29997,127.59653,117.0,충청북도 옥천군 옥천읍 옥천동이로 234,2010-12-17
197,210,충청북도,380959A001,충주시 달천동,36.95015,127.89728,76.0,충북 충주시 달천동 672-2,2010-08-19


In [4]:
DOC_TYPE = 'json'
API_URL = 'http://apis.data.go.kr/1390802/AgriWeather/WeatherObsrInfo/InsttWeather/getWeatherMonDayList'
YEAR = '2022'
MONTHS= [str(x).zfill(2) for x in range(1,13,1)]
API_KEY = '87r4QXW2oQcxjpXLQp6hKQe4aaKIme68yP2zGcxZuCWzDmobzuNxoBYvA5meOw5NMy7J/M8c/1fWtPdBFfWjZg=='


In [6]:
column_list = ['SpotCode','SpotName','Date','10cm','10Cmst','20cm','20Cmst','30cm','30Cmst']
res_cols = ['obsr_Spot_Code', 'obsr_Spot_Nm', 'date_Time', 'soil_Mitr_10',
            'soil_Mitr_10Cmst', 'soil_Mitr_20', 'soil_Mitr_20Cmst',
            'soil_Mitr_30', 'soil_Mitr_10Cmst']
res = pd.DataFrame(columns=column_list)

for m in MONTHS:
    for point, code in zip(regions['지점명'], regions['지점코드']):
        params ={'serviceKey' : API_KEY, 
         'Page_No' : '1', 
         'Page_Size' : '100',
         'search_Year' : YEAR,
         'search_Month' : m, 
         'obsr_Spot_Nm' : point,
         'obsr_Spot_Code' : code}
        
        response = requests.get(API_URL, params=params)
        # Check the status code

        # XML to DICT
        xmlToDict = xmltodict.parse(response.text)

        # DICT to JSON
        df_dict = json.loads(json.dumps(xmlToDict)) 
        print(df_dict['response']['header'])
        if 'result_Code' in df_dict['response']['header']:
            if df_dict['response']['header']['result_Code'] =='200' and df_dict['response']['body']['items'] is not None:
                for item in df_dict['response']['body']['items']['item']:
                    tmp_res= pd.DataFrame(item,index=[0])
                    tmp_res = tmp_res.reindex(columns=res_cols)
                    tmp_res.columns = column_list
                    res = pd.concat([res,tmp_res])
            else:
                print(df_dict['response']['body']['items'])


{'Result_Code': '301', 'Result_Msg': '요청 데이터 없음'}
{'result_Code': '200', 'result_Msg': 'OK'}
{'result_Code': '200', 'result_Msg': 'OK'}
{'result_Code': '200', 'result_Msg': 'OK'}
{'result_Code': '200', 'result_Msg': 'OK'}
{'Result_Code': '301', 'Result_Msg': '요청 데이터 없음'}
{'Result_Code': '301', 'Result_Msg': '요청 데이터 없음'}
{'result_Code': '200', 'result_Msg': 'OK'}
{'result_Code': '200', 'result_Msg': 'OK'}
{'result_Code': '200', 'result_Msg': 'OK'}
{'result_Code': '200', 'result_Msg': 'OK'}
{'result_Code': '200', 'result_Msg': 'OK'}
{'result_Code': '200', 'result_Msg': 'OK'}
{'result_Code': '200', 'result_Msg': 'OK'}
{'result_Code': '200', 'result_Msg': 'OK'}
{'result_Code': '200', 'result_Msg': 'OK'}
{'result_Code': '200', 'result_Msg': 'OK'}
{'result_Code': '200', 'result_Msg': 'OK'}
{'result_Code': '200', 'result_Msg': 'OK'}
{'result_Code': '200', 'result_Msg': 'OK'}
{'result_Code': '200', 'result_Msg': 'OK'}
{'result_Code': '200', 'result_Msg': 'OK'}
{'result_Code': '200', 'result_Ms

KeyError: 'response'

In [14]:
res = res.reset_index(drop=True)

In [16]:
del res['index']

In [19]:
res.to_csv('../results_data/06_토양수분.csv',encoding='cp949',index=False)